In [3]:
import sqlite3
import pandas as pd
import numpy as np
from gurobipy import *
import datetime
import time

In [4]:
#CONVERT DATA TO DATAFRAME

conn = sqlite3.connect("flight_sample.db")
c = conn.cursor()
query = c.execute('''SELECT * From flights ''')
cols = [column[0] for column in query.description]
results = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

results.to_csv('flights.csv')

#df = pd.read_sql_query("SELECT * FROM table_name", conn)
#dat = sqlite3.connect("flight.db")
#query = dat.execute("SELECT * From flight")
#cols = [column[0] for column in query.description]
#results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [5]:
#results.head(100)

In [6]:
# create dictionary of dataframes

origins = results['origin_id'].unique()

originsDict = {}
for i in origins:
    originsDict[i] = results[results['origin_id'] == i]

In [7]:
#originsDict

In [8]:
# create dictionary for optimization model
# format: {origin: destination1:{date1:price1, date2:price2},
#                  destination2:{date1:price1, date2:price2}}

X = {}
for i in originsDict.keys():
    X[i] = {}

for i in origins:
    for index, row in originsDict[i].iterrows():
        X[i][row['destination_id']] = {}
    for index, row in originsDict[i].iterrows():
        X[i][row['destination_id']][datetime.datetime.strptime(row['outbound_date'], "%Y-%m-%dT%H:%M:%S").date()]= \
        (datetime.datetime.strptime(row['inbound_date'], "%Y-%m-%dT%H:%M:%S").date(), row['price'])

# Automate the date intervals

In [9]:
user_start = '2017-05-01T00:00:00'
user_end = '2017-05-31T00:00:00'
#reads in the first and last day of the full timeframe of the user

try:
    
    dt_start = datetime.datetime.strptime(user_start, "%Y-%m-%dT%H:%M:%S").date()
    dt_end = datetime.datetime.strptime(user_end, "%Y-%m-%dT%H:%M:%S").date()
    
except ValueError:
    print ("Incorrect format")
    
delta = dt_end - dt_start
numdays = delta.days

date_list = [dt_start + datetime.timedelta(days=x) for x in range(0, numdays)]

#print(date_list[0])
#print(date_list[:5])
#delta.days
#dt_start

# Map date to nodes
#date_list = [dt_start + datetime.timedelta(days=x) for x in range(0, numdays)]
#date_map = {}
#date_map[datetime.date(2017, 5, 1)]

#date_list

# Create list of lists of dates

In [10]:
num_days = 10 #this should be specified by the user

dates = []
dates_list = []
for i in range(0,len(date_list)-num_days+1):
    dates = date_list[0:num_days]
    dates_list.append(dates)
    date_list.pop(0)
    
#dates_list

#All of the following code should be in one big loop
#that iterates through dates_list

# Create dummy dictionary

In [11]:
#create dummy dictionary of cities of interest
#month of May

Xcopy = X.copy()
Y = {}
key_list = origins

for i in Xcopy.keys():
    if i in key_list:
        Y[i] = Xcopy[i]

for i in Y.keys():
    for j in Y[i].keys():
        if j not in key_list:
            Y[i][j] = {}
    Y[i] = dict((k, v) for k, v in Y[i].iteritems() if v != {})  


In [12]:
#Y

In [13]:
date_list = dates_list[4]
#create dummy dictionary of dates of interest
Ycopy = Y.copy()
Z = {}
for i in Ycopy.keys():
    Z[i] = {}
    for j in Ycopy[i].keys():
        Z[i][j] = {}
        for k in Ycopy[i][j].keys():
            if k in date_list:
                Z[i][j][k] = Ycopy[i][j][k]
            else:
                Z[i][j][k] = {}
        Z[i][j] = dict((k, v) for k, v in Z[i][j].iteritems() if v != {})

In [14]:
#Z

# Separate source and non-source dictionaries

In [15]:
# source dictionary

source = 'ATL'

Zcopy = Z.copy()
S = {}

for i in Zcopy.keys():
    if i == source:
        S[source] = Zcopy[source]
    else:
        S[i] = {}

for i in S.keys():
    if source in Zcopy[i].keys():
        S[i][source] = Zcopy[i][source]

S = dict((k, v) for k, v in S.iteritems() if v != {})

In [16]:
# non-source dictionary

Zcopy2 = Z.copy()
Zcopy2[source] = {}
Zcopy2 = dict((k, v) for k, v in Zcopy2.iteritems() if v != {})
#with our current dataset, this will get rid of keys other than the source
#since we are missing so many flights

for i in Zcopy2.keys():
    if source in Zcopy2[i].keys():
        Zcopy2[i][source] = {}
    Zcopy2[i] = dict((k, v) for k, v in Zcopy2[i].iteritems() if v != {})
    
Zcopy2 = dict((k, v) for k, v in Zcopy2.iteritems() if v != {})

# Only one in- and out-bound date for source

In [17]:
dt_start = date_list[0] #datetime.date(2017, 5, 1)
dt_end = date_list[-1] #datetime.date(2017, 5, 30)
#this should come from the iteration:
#for day in dates_list: dt_start = day[0], dt_end = day[-1]

Scopy = S.copy()
T = {}

for i in Scopy.keys():
    T[i] = {}
    for j in Scopy[i].keys():
        T[i][j] = {}

for i in Scopy[source].keys():
    for j in Scopy[source][i].keys():
        if j == dt_start:
            T[source][i][j] = S[source][i][j]

for i in Scopy.keys():
    if i != source:
        for j in Scopy[i][source].keys():
            if j == dt_end:
                T[i][source][j] = Scopy[i][source][j]
                
sourceDict = T

In [18]:
nonsourceDict = dict(Zcopy2)

for i in nonsourceDict.keys():
    for j in nonsourceDict[i].keys():
        del nonsourceDict[i][j][dt_start]
        del nonsourceDict[i][j][dt_end]


In [19]:
#sourceDict

In [20]:
#nonsourceDict

# Create available cities and legs?

In [21]:
delta = dt_end - dt_start
numdays = delta.days
real_dates = [dt_start + datetime.timedelta(days=x) for x in range(1, numdays+1)]
real_cities = []
for key in nonsourceDict:
    real_cities.append(key)

In [22]:
#real_dates

In [23]:
sourceAvaiLegs = sourceDict.copy()
avaiLegs = nonsourceDict.copy()
SourceCity = source
firstDate = dt_start
lastDate = dt_end

In [24]:
#sourceAvaiLegs

In [25]:
#avaiLegs

# Optimization

In [26]:
# new model here
model = Model("WanderWorld")


# Add variables
# Now we have all the available source-related legs in 'sourceAvaiLegs'
# all the remaining available legs in 'avaiLegs' 
# all the inventory legs in 'invLegs' (well maybe we don't need this...)
# and they are all in the same format

legs = {} # available legs variables
cost = {} # corresponding cost

for origin, destDic in sourceAvaiLegs.iteritems():
    if(origin==SourceCity):
        for dest, dateDic in destDic.iteritems():
            for date, price in dateDic.iteritems():
                legs[origin, dest, date, price[0]+datetime.timedelta(days=1)] = model.addVar(
                    vtype=GRB.BINARY, name="Source_{}_to_{}_day_{}_{}".format(
                        origin, dest, date, price[0]))
                cost[origin, dest, date, price[0]+datetime.timedelta(days=1)] = price[1]
    else:
        for dest, dateDic in destDic.iteritems():
            for date, price in dateDic.iteritems():
                legs[origin, dest, date, price[0]+datetime.timedelta(days=1)] = model.addVar(
                    vtype=GRB.BINARY, name="Sink_{}_to_{}_day_{}_{}".format(
                        origin, dest, date, price[0]))
                cost[origin, dest, date, price[0]+datetime.timedelta(days=1)] = price[1]
                
for origin, destDic in avaiLegs.iteritems():
    for dest, dateDic in destDic.iteritems():
        for date, price in dateDic.iteritems():
            legs[origin, dest, date, price[0]+datetime.timedelta(days=1)] = model.addVar(
                vtype=GRB.BINARY, name="{}_to_{}_day_{}_{}".format(
                    origin, dest, date, price[0]))
            cost[origin, dest, date, price[0]+datetime.timedelta(days=1)] = price[1]

model.update()
print len(legs), len(cost)

avaiLegsByDest = {} # record all the available legs by Destination, no Inventory
avaiLegsByOri = {} # record all the available legs by Origin, no Inventory
for (origin, destination, depart, arri) in legs: # Without Inventory Variables
    try:
        avaiLegsByDest[destination].append((origin, destination, depart, arri))
    except:
        avaiLegsByDest[destination] = [(origin, destination, depart, arri)]
    try:
        avaiLegsByOri[origin].append((origin, destination, depart, arri))
    except:
        avaiLegsByOri[origin] = [(origin, destination, depart, arri)]

# Add the inventory arcs
# Inventory Variables
for date in real_dates:
    for city in real_cities:
        legs[city, city, date, date+datetime.timedelta(days=1)] = \
        model.addVar(vtype=GRB.BINARY, name="Inventory_{}_day_{}".format(
                    city, date))

model.update()

allLegsByDest = {} # record all the available legs by Destination
allLegsByOri = {} # record all the available legs by Origin
for (origin, destination, depart, arri) in legs: # Without Inventory Variables
    try:
        allLegsByDest[destination,arri].append((origin, destination, depart, arri))
    except:
        allLegsByDest[destination,arri] = [(origin, destination, depart, arri)]
    try:
        allLegsByOri[origin,depart].append((origin, destination, depart, arri))
    except:
        allLegsByOri[origin,depart] = [(origin, destination, depart, arri)]          

52 52


In [27]:
#legs

In [28]:
# Set Constraints
UpperBound = 10
LowerBound = 2 # can stay LowerBound + 1 nights

# all cities covered and visited only once
for destination in avaiLegsByDest:
    model.addConstr(quicksum(legs[origin, destination, depart, arri] 
                             for (origin,dest,depart,arri) in avaiLegsByDest[destination])== 1,
                   "VisitOnlyOnce_{}".format(destination))

# min and max number of days (inventory constraints)
for city in real_cities:
    model.addConstr(quicksum(legs[city, city, date, date+datetime.timedelta(days=1)] 
                             for date in real_dates) <= UpperBound,
                   "Stay_Upper_Bound_{}".format(city))
    model.addConstr(quicksum(legs[city, city, date, date+datetime.timedelta(days=1)] 
                             for date in real_dates) >= LowerBound,
                   "Stay_Lower_Bound_{}".format(city))

# Flow Balance
# Aux Arc balance
SourceFlow = quicksum(legs[origin, destination, depart, arri]
                        for (origin, destination, depart, arri) in allLegsByOri[SourceCity,firstDate])
SinkFlow = quicksum(legs[origin, destination, depart, arri]
                        for (origin, destination, depart, arri) in allLegsByDest[SourceCity,lastDate+datetime.timedelta(days=1)])
model.addConstr(SourceFlow==1, "SourceOneOut")
model.addConstr(SinkFlow==1, "SinkOneIn")

# Ordinary balance
for city in real_cities:
    for date in real_dates:
        if(date==firstDate):
            continue
        else:
            In = 0
            Out = 0
            if(allLegsByDest.has_key((city,date))):
                In = quicksum(legs[origin, destination, depart, arri] 
                                     for (origin, destination, depart, arri) in allLegsByDest[city,date])
            else:
                ''
            if(allLegsByOri.has_key((city,date))):
                Out = quicksum(legs[origin, destination, depart, arri] 
                                     for (origin, destination, depart, arri) in allLegsByOri[city,date])
            else:
                ''
        model.addConstr(In == Out, "FlowBalance_{}_day_{}".format(city,date))

model.update()

In [29]:
#real_dates

In [30]:
# Set Objectives
obj = quicksum(cost[key]*legs[key] for key in cost)

model.setObjective(obj, GRB.MINIMIZE)

model.update()

model.optimize()

Optimize a model with 39 rows, 79 columns and 261 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e+01, 3e+03]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+01]
Presolve removed 39 rows and 79 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 3.928000000000e+03, best bound 3.928000000000e+03, gap 0.0%


# Format output

In [32]:
#initialize main list

routes = []
route_tuples = []

for (key,value) in legs.iteritems():
    if (value.x!=0) and key[0] != key[1]:
        #print key, cost[key], str(key[2])[-2:]
        route_tuples.append(key+(cost[key],))

print ""

#print route_tuples
        
route_tuples = sorted(route_tuples, key=lambda tup: tup[2]) #sorts on date
route_tuples.append(model.objVal) #adds price to list
route_tuples = tuple(route_tuples) #transforms list to tuple

#for i in route_tuples:
    #print i

In [33]:
#in main loop:
routes.append(route_tuples)

#sort and slice at very end:
sorted_routes = sorted(routes, key=lambda tup: tup[-1]) #sorts on total price
final_results = sorted_routes[0:4] #or however many we want to return

In [34]:
final_results

[(('ATL', u'LON', datetime.date(2017, 5, 5), datetime.date(2017, 5, 6), 445.0),
  (u'LON', u'PMI', datetime.date(2017, 5, 8), datetime.date(2017, 5, 9), 60.0),
  (u'PMI',
   u'CDG',
   datetime.date(2017, 5, 11),
   datetime.date(2017, 5, 12),
   71.0),
  (u'CDG',
   'ATL',
   datetime.date(2017, 5, 14),
   datetime.date(2017, 5, 15),
   3352.0),
  3928.0)]